# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

In [2]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
df = pd.read_csv(url)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('unknown')
    else:
        df[col] = df[col].fillna(df[col].mean())

def onehot_encode(arr):
    uniques = list(set(arr))
    mapping = {val: i for i, val in enumerate(uniques)}
    res = np.zeros((len(arr), len(uniques)), dtype=int)
    for i, val in enumerate(arr):
        res[i, mapping[val]] = 1
    return res

def minmax_scale(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())

cat_mask = (df.dtypes == "object").values
cat_cols = df.columns[cat_mask]
num_cols = [c for c in df.columns if not df[c].dtype == "object" and c != 'Claim']

cat_encoded = [onehot_encode(df[col].values) for col in cat_cols]
cat_en = np.hstack(cat_encoded) if cat_encoded else np.array([])

num_scaled = [minmax_scale(df[col].values) for col in num_cols]
num_scaled = np.column_stack(num_scaled) if num_scaled else np.array([])

# Собираем X
if cat_en.size and num_scaled.size:
    X = np.hstack([cat_en, num_scaled])
elif cat_en.size:
    X = cat_en
else:
    X = num_scaled

# Целевая переменная
y = df['Claim'].values

print("X:", X.shape)
print("y :", y.shape)


X: (63326, 204)
y : (63326,)


Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели. 

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [3]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
df = pd.read_csv(url)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('unknown')
    else:
        df[col] = df[col].fillna(df[col].mean())

# downsampling
mj = df[df['Claim'] == 0]
mn = df[df['Claim'] == 1]

mj_down = mj.sample(len(mn), random_state=42)
mn_dowm = pd.concat([mj_down, mn]).reset_index(drop=True)

def onehot_encode(arr):
    uniques = list(set(arr))
    mapping = {v: i for i, v in enumerate(uniques)}
    res = np.zeros((len(arr), len(uniques)), dtype=int)
    for i, v in enumerate(arr):
        res[i, mapping[v]] = 1
    return res


cat_cols = [c for c in df.columns if df[c].dtype == 'object']
cat_selected = []
for c in cat_cols:
    if df[c].nunique() <= 20:
        cat_selected.append(c)
    else:
        top_values = df[c].value_counts().nlargest(10).index
        df[c] = df[c].apply(lambda x: x if x in top_values else 'Other')
        cat_selected.append(c)

cat_encoded = [onehot_encode(df[c].values) for c in cat_selected]
cat_encoded = np.hstack(cat_encoded) if cat_encoded else np.array([])

#Масштабирование
num_cols = [c for c in df.columns if df[c].dtype != 'object' and c != 'Claim']
def minmax_scale(arr):
    arr = np.array(arr, dtype=float)
    return (arr - arr.min()) / (arr.max() - arr.min())

num_scaled = [minmax_scale(df[c].values) for c in num_cols]
num_scaled = np.column_stack(num_scaled) if num_scaled else np.array([])

if cat_encoded.size and num_scaled.size:
    X = np.hstack([cat_encoded, num_scaled])
elif cat_encoded.size:
    X = cat_encoded
else:
    X = num_scaled

y = df['Claim'].values

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (63326, 51)
y shape: (63326,)


### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
df = pd.read_csv(url)

y = np.where(df['Claim'] == 'Yes', 1, 0)

for col in df.columns:
    if df[col].dtype == 'object' and col != 'Claim':
        df[col] = df[col].fillna('unknown')
    elif col != 'Claim':
        df[col] = df[col].fillna(df[col].mean())

cat_cols = [c for c in df.columns if df[c].dtype == 'object' and c != 'Claim']
num_cols = [c for c in df.columns if df[c].dtype != 'object' and c != 'Claim']

def onehot_encode(arr):
    uniques = list(set(arr))
    mapping = {v:i for i,v in enumerate(uniques)}
    res = np.zeros((len(arr), len(uniques)), dtype=int)
    for i, v in enumerate(arr):
        res[i, mapping[v]] = 1
    return res

cat_encoded = [onehot_encode(df[c].values) for c in cat_cols]
cat_encoded = np.hstack(cat_encoded) if cat_encoded else np.array([])

def minmax_scale(arr):
    arr = np.array(arr, dtype=float)
    return (arr - arr.min()) / (arr.max() - arr.min())

num_scaled = [minmax_scale(df[c].values) for c in num_cols]
num_scaled = np.column_stack(num_scaled) if num_scaled else np.array([])

if cat_encoded.size and num_scaled.size:
    X = np.hstack([cat_encoded, num_scaled])
elif cat_encoded.size:
    X = cat_encoded
else:
    X = num_scaled

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_class = np.round(y_pred).astype(int)

#Проверяем точность
accuracy = (y_pred_class == y_test).mean()
print("Accuracy:", accuracy)


ModuleNotFoundError: No module named 'sklearn'

Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

NameError: name 'X_bias' is not defined

In [ ]:
# посчитать аналитическое решение с регуляризацией

Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


url = "https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv"
df = pd.read_csv(url)


y = np.where(df['Claim'] == 'Yes', 1, 0)


for col in df.columns:
    if df[col].dtype == 'object' and col != 'Claim':
        df[col] = df[col].fillna('unknown')
    elif col != 'Claim':
        df[col] = df[col].fillna(df[col].mean())


cat_cols = [c for c in df.columns if df[c].dtype == 'object' and c != 'Claim']
num_cols = [c for c in df.columns if df[c].dtype != 'object' and c != 'Claim']

def encode_cat(arr):
    uniques = list(set(arr))
    mapping = {v:i for i,v in enumerate(uniques)}
    mat = np.zeros((len(arr), len(uniques)))
    for i, v in enumerate(arr):
        mat[i, mapping[v]] = 1
    return mat

cat_encoded = [encode_cat(df[c].values) for c in cat_cols]
cat_encoded = np.hstack(cat_encoded) if cat_encoded else np.array([])


def scale_num(arr):
    arr = np.array(arr, dtype=float)
    return (arr - arr.min()) / (arr.max() - arr.min())

num_scaled = [scale_num(df[c].values) for c in num_cols]
num_scaled = np.column_stack(num_scaled) if num_scaled else np.array([])

if cat_encoded.size and num_scaled.size:
    X = np.hstack([cat_encoded, num_scaled])
elif cat_encoded.size:
    X = cat_encoded
else:
    X = num_scaled

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)




In [41]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error на тестовой выборке:", mse)# посчитайте MSE, предварительно округлив предсказанные ответы до целого

### Вывод (1 балла)

Я заполнил пустые значения в числовых колонках средними, а в текстовых — пометкой 'unknown'. Категориальные признаки перевёл в числа, числовые масштабировал, чтобы всё было сопоставимо. Линейная модель предсказывает довольно неплохо, но так как это задача классификации, точность могла бы быть лучше, если бы использовались дополнительные метрики или методы балансировки классов.